In [1268]:
import pandas as pd
import chardet
import os
import re
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pandas import Period

pd.set_option("display.max_columns", None)

### Tool

In [1269]:
all_columns = ["ASIN", "Nom du produit", "Marque", "COGS", "COGS evol LM", "COGS evol SPLY", "Unités", "Unités evol LM", "Unités evol SPLY", "Retours", "Retours evol LM", "Retours evol SPLY", "COGS LM", "Unités LM", "Retours LM", "is_in_vendor_central", "Item no.", "EAN", "Item", "Extras", "Size", "Description Item", "Cat. 1", "Cat. 2", "Cat. 3", "Cat. 4", "Visible", "Title Content", "Variations", "Brand store URL", "Images", "Videos", "AI summary", "Code", "Catalogue", "Ads Impressions CM", "Ads Clicks CM", "Ads CTR CM", "Ads Units 14d CM", "Ads Costs CM", "Ads RoAS CM", "Ads CVR CM", "Sell-out CM", "Sell-out PM", "Total sell-out CM", "Total sell-out PM", "Revenue CM", "Revenue PM", "Units CM", "Units PM", "SRP", "Margin", "Replacements CM", "Replacements PM", "Stock", "Stock value", "Total stock", "Total stock value", "Reviews", "Stars", "Coverage", "Rank 1", "Buy Box", "Buy Box PM", "Days not Buy Box", "Views CM", "Views PM", "CVR CM", "CVR PM", "is_in_amvisor", "extract_date"]

In [1270]:
restricted = ['ASIN', "Nom du produit", "EAN", "COGS", 'Unités', 'Retours', 'is_in_vendor_central', 'is_in_amvisor', 'Cat. 1', 'Cat. 2', 'Cat. 3', 'Cat. 4']

In [1271]:
interesting_columns = ["ASIN", "Nom du produit", "COGS", "COGS SPLY (€)", "COGS diff SPLY (€)", "COGS SPLY (%)", "Unités", "SRP"]

### Load data

In [1272]:
df = pd.read_csv("../data/all_data.csv", dtype={"AI summary": str, "EAN": str}, low_memory=False)
df = df.sort_values(["extract_date", "COGS"], ascending=False)
print(df.shape)
df.head(3)

(34147, 74)


,ASIN,Nom du produit,Marque,COGS,COGS LM (%),COGS SPLY (%),Unités,Unités LM (%),Unités SPLY (%),Retours,Retours LM (%),Retours SPLY (%),Source (Fabrication ou Approvisionnement),is_in_vendor_central,Item no.,EAN,Item,Extras,Size,Description Item,Cat. 1,Cat. 2,Cat. 3,Cat. 4,Visible,Title Content,Variations,Brand store URL,Images,Videos,AI summary,Code,Catalogue,Ads Impressions CM,Ads Clicks CM,Ads CTR CM,Ads Units 14d CM,Ads Costs CM,Ads RoAS CM,Ads CVR CM,Sell-out CM,Sell-out PM,Total sell-out CM,Total sell-out PM,Units CM,Units PM,SRP,Margin,Replacements CM,Replacements PM,Stock,Stock value,Total stock,Total stock value,Reviews,Avg. stars,Stars,Coverage,Rank 1,Buy Box,Buy Box PM,Days not Buy Box,Views CM,Views PM,CVR CM,CVR PM,is_in_amvisor,Net PPM,Net PPM SPLY,extract_date,annee,mois,COGS SPLY (€),COGS diff SPLY (€)
23,B00005QFCJ,Hama Adaptateur Cassette VHS-C/VHS (Convertiss...,Hama,10142.55,-7.30,63.37,495.0,-7.30,63.37,64.0,-31.91,45.45,1_fab,True,00044704,4007249447043,Hama Cassette adaptatrice VHS-C/VHS motorisée,NaN,NaN,REGULAR,Photo & Accessories,Studio & Diverse,Camera,Adapters,Yes,Hama Adaptateur Cassette VHS-C/VHS (Convertiss...,B00005QFCJ,NaN,13.0,2.0,Les clients apprécient le bon rapport qualité-...,PR,STO,66234.0,1550.0,2.3402,225.0,200.90,27.36,14.52,0.00,0.00,9302.46,10941.66,0.0,0.0,41.99,19.8,0.0,0.0,0.0,0.0,293.0,6003.57,901.0,4.0,3926.0,NaN,1304.0,Amazon,100.0,NaN,8037.0,8960.0,5.65,5.96,True,39.35,42.289092,2025-03,2025,3,6208.47,3934.08
82,B00006JAQ7,"Hama 007103 - Ruban photo transparent, Blanc, ...",Hama,3787.44,105.46,851.83,1313.0,102.94,1041.74,11.0,1000.00,NaN,1_fab,True,00007103,4007249071033,Hama Distributeur de bandes adhésives,NaN,2 x 500 bandes,REGULAR,Picture World,Storage & Design,Photo Storage & Presentation,Storage & Archiving,Yes,"Hama 007103 - Ruban photo transparent, Blanc, ...",B00005YVIU B00006JAQ7 B0CPF2BMJL,NaN,5.0,2.0,Les clients apprécient la super adhérence des ...,PR,STO,90395.0,1144.0,1.2656,552.0,526.34,4.13,48.25,884.83,394.44,3509.31,1843.40,305.0,136.0,3.99,68.1,0.0,0.0,356.0,1032.4,786.0,2278.34,2000.0,4.6,12826.0,1.1,398.0,Amazon,58.8,NaN,2642.0,1730.0,46.06,37.40,True,35.85,52.236631,2025-03,2025,3,397.91,3389.53
32,B00005YVIU,"Hama Distributeurs Coins photo (1000 coins, au...",Hama,3035.01,114.29,7188.69,1044.0,113.93,8600.00,9.0,NaN,NaN,1_fab,True,00007108,4007249071088,Hama Boîte distributrice coins adhésifs pr photos,Transparent,2x500 soins,REGULAR,Picture World,Storage & Design,Photo Storage & Presentation,Storage & Archiving,Yes,"Hama Distributeurs Coins photo (1000 coins, au...",B00005YVIU B00006JAQ7 B0CPF2BMJL,NaN,7.0,1.0,Les clients apprécient la super adhérence des ...,PR,PUA,184208.0,1878.0,1.0195,738.0,1144.90,2.76,39.30,1058.50,124.70,2834.91,1416.30,365.0,43.0,3.99,70.2,0.0,0.0,167.0,484.3,552.0,1600.80,2002.0,4.6,12827.0,0.4,273.0,Amazon,58.8,NaN,2494.0,1249.0,39.09,39.07,True,38.82,54.560787,2025-03,2025,3,41.64,2993.37


## Exploration

##### Sell-out par mois

In [1273]:
# Agrégation COGS total par mois
cogs_by_month = df.groupby(df['extract_date'])['COGS'].sum().reset_index()
cogs_by_month.columns = ['Mois', 'Total COGS']

cogs_by_month

,Mois,Total COGS
0,2024-01,179416.82
1,2024-02,137340.90
2,2024-03,123239.89
3,2024-04,110183.91
4,2024-05,106747.82
5,2024-06,116149.04
6,2024-07,97924.40
7,2024-08,88167.40
8,2024-09,92968.87
9,2024-10,89723.16


In [1274]:
total = df.groupby('extract_date')['COGS'].sum().reset_index()
fig1 = px.line(total, x='extract_date', y='COGS', title='Sell-out Total (COGS)')
fig1.show()

In [1275]:
# Agréger les COGS par mois et année
cogs_by_month = df.groupby(['annee', 'mois'])['COGS'].sum().reset_index()

# Graphique interactif
fig = px.line(
    cogs_by_month,
    x='mois',
    y='COGS',
    color='annee',
    markers=True,
    labels={'mois': 'Mois', 'COGS': 'Total COGS (€)', 'annee': 'Année'},
    title='Évolution mensuelle du COGS par année'
)

fig.update_layout(
    xaxis=dict(dtick=1),
    yaxis=dict(tickformat=".0f"),
    legend_title_text='Année'
)

fig.show()

##### Ventes hors catalogue

In [1276]:
# Produits vendus dans Vendor Central mais pas dans l'assortiment Amvisor
hors_catalogue = df[df['is_in_vendor_central'] & ~df['is_in_amvisor'] & df['COGS'].notna()]

print(f"Nombre de produits vendus hors catalogue sur les mois d'analyse: {hors_catalogue.ASIN.nunique()}")
hors_catalogue[restricted].head(3)

Nombre de produits vendus hors catalogue sur les mois d'analyse: 1476


,ASIN,Nom du produit,EAN,COGS,Unités,Retours,is_in_vendor_central,is_in_amvisor,Cat. 1,Cat. 2,Cat. 3,Cat. 4
35,B00006IVA9,Hama CD de nettoyage (pour tête de lecture las...,NaN,516.02,98.0,2.0,True,False,NaN,NaN,NaN,NaN
414,B003PFBLOQ,"Hama Horloge murale ""PP-245"" (radio-pilotée DC...",NaN,430.56,24.0,4.0,True,False,NaN,NaN,NaN,NaN
117,B00006JDOB,Hama Étuis pour 20 diapositives avec cadre au ...,NaN,139.74,17.0,NaN,True,False,NaN,NaN,NaN,NaN


##### Evolution des categories

In [1277]:
category = "Cat. 1"
cat_cogs = df.groupby(['extract_date', category])['COGS'].sum().reset_index()
cat_cogs_pivot = cat_cogs.pivot(index=category, columns='extract_date', values='COGS')
cat_cogs_pivot

extract_date,2024-04,2024-05,2024-06,2024-07,2024-08,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03
Cat. 1,,,,,,,,,,,
-,NaN,0.00,0.00,0.00,0.00,37.34,57.37,NaN,NaN,NaN,0.00
Audio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11081.95,7751.08,5343.28,5305.20
Cables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1602.96,2267.61,1630.58,1835.22
Computing & Tablet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2127.78,2737.82,1999.19,2563.80
Consumer Electronics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25610.37,26234.74,23112.28,22761.33
Distribution Brand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,579.80,457.07,761.76,77.39
Gaming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.00,146.47,164.43,56.27
Home & Office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12625.67,11896.32,8488.69,12091.93
Home & Outdoor,18936.64,17151.79,28016.46,13200.06,14337.57,16209.83,15687.75,479.09,377.96,114.43,90.66


In [1278]:
cat = df.groupby(['extract_date', 'Cat. 2'])['COGS'].sum().reset_index()
fig2 = px.line(cat, x='extract_date', y='COGS', color='Cat. 2', title='Sell-out par Catégorie')
fig2.show()

##### Produits en baisse durant Q1

In [1279]:
# On garde les produits avec données sur plusieurs mois du Q1
pivot_cogs = df.pivot_table(index='ASIN', columns='extract_date', values='COGS', aggfunc='sum')

# Calcul de la baisse entre chaque mois
pivot_cogs['delta_dec'] = pivot_cogs["2024-12"] - pivot_cogs["2024-11"]
pivot_cogs['delta_jan'] = pivot_cogs["2025-01"] - pivot_cogs["2024-12"]
pivot_cogs['delta_feb'] = pivot_cogs["2025-02"] - pivot_cogs["2025-01"]
pivot_cogs['delta_mar'] = pivot_cogs["2025-03"] - pivot_cogs["2025-02"]

pivot_cogs['delta_dec_perc'] = (pivot_cogs["2024-12"] - pivot_cogs["2024-11"])/pivot_cogs["2024-11"]
pivot_cogs['delta_jan_perc'] = (pivot_cogs["2025-01"] - pivot_cogs["2024-12"])/pivot_cogs["2024-12"]
pivot_cogs['delta_feb_perc'] = (pivot_cogs["2025-02"] - pivot_cogs["2025-01"])/pivot_cogs["2025-01"]
pivot_cogs['delta_mar_perc'] = (pivot_cogs["2025-03"] - pivot_cogs["2025-02"])/pivot_cogs["2025-02"]

# On identifie les ASINs en forte baisse (< -20%)
pivot_cogs['baisse_detectée'] = (
    # (pivot_cogs['delta_dec_perc'] < -0.2) |
    # (pivot_cogs['delta_jan_perc'] < -0.2) | 
    (pivot_cogs['delta_feb_perc'] < -0.2) | 
    (pivot_cogs['delta_mar_perc'] < -0.2)
    
)

# Tri pour voir les pires baisses
produits_baisse = pivot_cogs[pivot_cogs['baisse_detectée']].sort_values(by='delta_mar')

# Extraire les noms uniques associés aux ASIN
noms = df[['ASIN', 'Nom du produit']].drop_duplicates()

# Fusion avec ton tableau pivot
produits_baisse = produits_baisse.reset_index().merge(noms, on='ASIN', how='left')
produits_en_baisse_ASIN = produits_baisse['ASIN'].unique()

print(f"{len(produits_en_baisse_ASIN)} produits ayat subit une baisse > 20% durant la periode d'analyse")
produits_baisse.head(10)

681 produits ayat subit une baisse > 20% durant la periode d'analyse


,ASIN,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,delta_dec,delta_jan,delta_feb,delta_mar,delta_dec_perc,delta_jan_perc,delta_feb_perc,delta_mar_perc,baisse_detectée,Nom du produit
0,B08B3QTPMF,0.00,100.98,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,1009.80,0.00,0.00,0.00,1009.80,-1009.80,NaN,NaN,inf,-1.000000,True,Hama Support TV Design Chevalet pour téléviseu...
1,B08B3QTPMF,0.00,100.98,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,1009.80,0.00,0.00,0.00,1009.80,-1009.80,NaN,NaN,inf,-1.000000,True,NaN
2,B0764LMPBN,3778.07,3116.06,734.53,590.52,1388.24,795.42,300.44,1118.88,2134.16,1999.48,2363.23,1853.41,2103.08,1916.60,1108.52,-509.82,249.67,-186.48,-808.08,-0.215730,0.134708,-0.088670,-0.421622,True,Hama Multiprise avec Interrupteur 10 Prises 16...
3,B00A44OC0G,2178.98,957.85,506.30,536.52,212.27,1469.51,993.43,1529.14,2576.65,2460.73,1461.47,619.87,1041.48,978.36,210.40,-841.60,421.61,-63.12,-767.96,-0.575859,0.680159,-0.060606,-0.784946,True,Hama Barre Multiprise 6 Prises avec Interrupte...
4,B08GSRV3PB,1398.60,2798.40,599.40,0.00,0.00,12116.77,33.30,965.70,1228.54,432.90,932.52,0.00,99.90,666.36,33.32,-932.52,99.90,566.46,-633.04,-1.000000,inf,5.670270,-0.949997,True,Hama Tête Thermostatique Connectée et Intellig...
5,B097HN8V5V,1288.94,1741.92,1285.92,291.84,294.88,921.12,1211.47,1169.62,2717.47,2698.51,579.31,81.89,621.63,560.89,6.06,-497.42,539.74,-60.74,-554.83,-0.858642,6.591037,-0.097711,-0.989196,True,Hama Câble d'Antenne TV (Cable TV Coaxial Mâle...
6,B07HY9LK8J,NaN,NaN,NaN,9.60,0.00,0.00,28.80,86.40,441.60,1545.60,201.60,38.40,28.80,412.80,0.00,-163.20,-9.60,384.00,-412.80,-0.809524,-0.250000,13.333333,-1.000000,True,"Rapoo Clavier multimédia USB Filaire ""N2400"" (..."
7,B07HY9LK8J,NaN,NaN,NaN,9.60,0.00,0.00,28.80,86.40,441.60,1545.60,201.60,38.40,28.80,412.80,0.00,-163.20,-9.60,384.00,-412.80,-0.809524,-0.250000,13.333333,-1.000000,True,NaN
8,B08GS8FVB5,537.94,1096.57,972.43,413.80,579.32,537.94,240.51,553.36,255.99,276.68,1223.06,1587.65,2903.83,783.48,385.28,364.59,1316.18,-2120.35,-398.20,0.298097,0.829011,-0.730191,-0.508245,True,Hama Destructeur de Documents Basic S6 (Broyeu...
9,B08HQH5PSV,298.97,427.10,384.39,589.81,587.10,707.10,240.00,360.00,490.84,360.00,202.71,120.00,120.00,560.00,240.00,-82.71,0.00,440.00,-320.00,-0.408021,0.000000,3.666667,-0.571429,True,Hama Horloge Murale Numérique Jumbo (Radio-pil...


##### Produits en baisse Same Period Last Year

In [1280]:
dt = df
dt = dt.sort_values(["COGS diff SPLY (€)"], ascending=True)[["ASIN", "Nom du produit", "COGS", "COGS SPLY (€)", "COGS diff SPLY (€)", "COGS SPLY (%)", "Unités", "SRP"]]
dt.head(50)

,ASIN,Nom du produit,COGS,COGS SPLY (€),COGS diff SPLY (€),COGS SPLY (%),Unités,SRP
6958,B0BL1SS1WK,Hama MW-300 V2 Souris sans Fil - Technologie 2...,35.71,6223.48,-6187.77,-99.43,5.0,NaN
166,B0000AIRY7,Hama Cadre en plastique Madrid (taille de cadr...,108.42,3536.16,-3427.74,-96.93,26.0,9.49
6853,B08YS1PZWL,Hama Adaptateur Jack 6.35 vers 3.5 Garantie 10...,343.80,3321.01,-2977.21,-89.65,191.0,2.99
4015,B09MQQM1ZJ,Xavax Socle machine à laver avec étagère (Supp...,419.40,3335.63,-2916.23,-87.43,8.0,109.99
697,B01N6NQ73J,Thomson Casque TV avec câble extra long (Casqu...,892.08,3741.80,-2849.72,-76.16,82.0,19.99
3683,B01N6NQ73J,Thomson Casque TV avec câble extra long (Casqu...,980.35,3575.48,-2595.13,-72.58,89.0,19.99
6889,B09MQQM1ZJ,Xavax Socle machine à laver avec étagère (Supp...,728.00,3016.00,-2288.00,-75.86,14.0,109.99
5901,B00006JAQ7,"Hama 007103 - Ruban photo transparent, Blanc, ...",433.77,2680.28,-2246.51,-83.82,150.0,3.99
6197,B0026MT4CK,Xavax Tuyau d'Arrivée d'Eau Froide 2.5m (Tuyau...,51.21,2227.66,-2176.45,-97.70,9.0,11.49
5863,B00006J4EP,Hama Boîtier standard (pour CD et disques Blu-...,101.17,2246.97,-2145.80,-95.50,27.0,8.99


##### Product Analysis

In [1281]:
dt = df

# 1. Produits hors catalogue
hors_catalogue = dt[~dt['is_in_amvisor']]

# 2. Évolution du nombre de reviews
reviews_evolution = dt.groupby(['ASIN', 'extract_date'])['Reviews'].sum().unstack(fill_value=0)

# 3. Évolution de la note moyenne (Stars)
stars_evolution = dt.groupby(['ASIN', 'extract_date'])['Avg. stars'].mean().unstack()

# 4. Évolution du nombre de retours
retours_evolution = dt.groupby(['ASIN', 'extract_date'])['Retours'].sum().unstack(fill_value=0)

# 5. Évolution du CVR
cvr_evolution = dt.groupby(['ASIN', 'extract_date'])['CVR CM'].mean().unstack()

# 6. Évolution des ventes
sellout_evolution = dt.groupby(['ASIN', 'extract_date'])['COGS'].sum().unstack(fill_value=0)

# 7. Évolution du status
status_evolution = dt.groupby(['ASIN', 'extract_date'])['Code'].first().unstack()

# 8. Ads
ads_imp_evolution = dt.groupby(['ASIN', 'extract_date'])['Ads Impressions CM'].sum().unstack(fill_value=0)

# 9. Evolution du margin
margin_evolution = dt.groupby(['ASIN', 'extract_date'])['Margin'].first().unstack()

In [1282]:
product = 'B0000AISOV'
df[df.ASIN == product]

reviews_evolution[reviews_evolution.index == product]
stars_evolution[stars_evolution.index == product]
status_evolution[status_evolution.index == product]
ads_imp_evolution[ads_imp_evolution.index == product]
margin_evolution[margin_evolution.index == product]

extract_date,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03
ASIN,,,,,,,,,,,,,,,
B0000AISOV,NaN,NaN,NaN,NaN,37.7,37.7,76.0,79.9,NaN,80.5,78.0,74.4,79.2,77.9,73.0


##### Top products

TOP 20 - Overall sales

In [1283]:
top_products_overall = df.groupby('ASIN')['COGS'].sum().sort_values(ascending=False).head(20)
top_products_overall = top_products_overall.index.tolist()
print(top_products_overall)

['B00005QFCJ', 'B01N6NQ73J', 'B00D2S92L0', 'B0BNLLF6YV', 'B0764LMPBN', 'B0009V6Q30', 'B08GSRV3PB', 'B00EUSMHE0', 'B00006J4EP', 'B003PFBLOQ', 'B00006JAXW', 'B00A44OC0G', 'B07GLF5SMF', 'B097HN8V5V', 'B00CRB69MI', 'B09MQQM1ZJ', 'B01N2RIZMU', 'B00SL6I2WI', 'B0000AI2SY', 'B0053DXXJO']


TOP 80/20 - Pareto sales

In [1284]:
# 1. Regrouper les COGS totaux par ASIN
cogs_total = df.groupby('ASIN')['COGS'].sum().sort_values(ascending=False)

# 2. Calcul du % cumulatif
cogs_cum = cogs_total.cumsum() / cogs_total.sum()

# 3. Repérer les ASIN qui composent les 80% du COGS total
pareto_asin = cogs_cum[cogs_cum <= 0.80].index.tolist()

# Optionnel : nombre d'ASIN qui génèrent 80% du COGS
nb_top = len(pareto_asin)
pourcentage = nb_top / len(cogs_total)

print(f"{nb_top} ASIN (~{pourcentage:.1%}) génèrent 80% du COGS total.")

234 ASIN (~6.5%) génèrent 80% du COGS total.


In [1285]:
# Récupérer les noms uniques des produits concernés
top_products = df[df['ASIN'].isin(pareto_asin)][['ASIN', 'Nom du produit']].drop_duplicates()

# Tri facultatif pour mettre les plus gros vendeurs en haut
top_products = top_products.merge(cogs_total.rename("COGS"), on="ASIN").sort_values(by="COGS", ascending=False)

top_products.head(20)

,ASIN,Nom du produit,COGS
0,B00005QFCJ,Hama Adaptateur Cassette VHS-C/VHS (Convertiss...,96177.95
20,B01N6NQ73J,Thomson Casque TV avec câble extra long (Casqu...,36337.06
4,B00D2S92L0,Xavax Hama Boîte de transport pour un gâteau (...,28794.72
7,B0BNLLF6YV,Hama iTuner Hi-FI DIT2105SBTX Dab+ Bluetooth (...,25818.83
13,B0764LMPBN,Hama Multiprise avec Interrupteur 10 Prises 16...,25300.64
8,B0009V6Q30,"Hama Boîtier Slim (pour CD et disques Blu-ray,...",22159.83
186,B08GSRV3PB,Hama Tête Thermostatique Connectée et Intellig...,21305.71
223,B00EUSMHE0,THOMSON Kit Casque sans Fil TVWHP3203D (on-Ear...,21195.16
17,B00006J4EP,Hama Boîtier standard (pour CD et disques Blu-...,20882.53
40,B003PFBLOQ,"Hama Horloge murale ""PP-245"" (radio-pilotée DC...",19459.52


In [1286]:
import plotly.express as px

df_pareto = cogs_total.reset_index(name='COGS')
df_pareto['COGS_cum_pct'] = df_pareto['COGS'].cumsum() / df_pareto['COGS'].sum()

fig = px.line(df_pareto, y='COGS_cum_pct', title="Courbe de Pareto (COGS)")
fig.update_layout(xaxis_title="Produits (triés)", yaxis_title="COGS cumulé (%)")
fig.show()


TOP 80/20 - Pareto par mois

In [1287]:
# Initialiser un dictionnaire pour stocker les résultats 80/20 par mois
pareto_results = {}

# Extraire la liste des mois uniques
mois_uniques = sorted(df['extract_date'].unique())

# Appliquer le principe 80/20 pour chaque mois
for mois in mois_uniques:
    df_mois = df[df['extract_date'] == mois]
    cogs_mensuel = df_mois.groupby('ASIN')['COGS'].sum().sort_values(ascending=False)
    cogs_cum = cogs_mensuel.cumsum() / cogs_mensuel.sum()
    top_asin = cogs_cum[cogs_cum <= 0.80].index.tolist()
    
    pareto_results[str(mois)] = {
        'nb_top_asin': len(top_asin),
        'total_asin': len(cogs_mensuel),
        'percent_top_asin': len(top_asin) / len(cogs_mensuel) if len(cogs_mensuel) > 0 else 0,
        'top_asin_list': top_asin
    }

# Convertir en DataFrame pour affichage synthétique
pareto_summary = pd.DataFrame([
    {
        "Mois": k,
        "Nb ASIN 80% COGS": v["nb_top_asin"],
        "Total ASIN": v["total_asin"],
        "% ASIN dans les 80%": round(v["percent_top_asin"] * 100, 2)
    }
    for k, v in pareto_results.items()
])

pareto_summary

,Mois,Nb ASIN 80% COGS,Total ASIN,% ASIN dans les 80%
0,2024-01,179,1468,12.19
1,2024-02,187,1444,12.95
2,2024-03,186,1411,13.18
3,2024-04,186,2464,7.55
4,2024-05,192,2522,7.61
5,2024-06,156,2566,6.08
6,2024-07,167,2550,6.55
7,2024-08,167,2562,6.52
8,2024-09,162,1254,12.92
9,2024-10,164,2494,6.58
